In [1]:
!pip3 install catboost
!pip3 install optuna
!pip3 install shap

In [2]:
import numpy as np
import pandas as pd
import optuna
import shap
from optuna.samplers import TPESampler
import catboost
from catboost.utils import eval_metric

from sklearn.model_selection import train_test_split

In [5]:
train_df = pd.read_parquet('/kaggle/input/ozon-hack-4/merged_df (1).parquet')
train_df.head()

,target,characteristic_attributes_mapping1,color1,type1,cat21,cat41,main_pic_embeddings_resnet_v11,description1,name_bert_641,characteristic_attributes_mapping2,...,brand_eq,country_eq,attr_dis,color_dis,type_dis,cat2_dis,cat4_dis,pic_dis,description_dis,name_dis
0,0,"[-0.43684134, 0.12223242, -0.37675297, 0.76652...","[1.3241816, -0.20996426, 0.41126335, 0.0240248...","[2.0598493, -0.009100158, 0.18228978, -0.16186...","[2.0046031, -0.5959641, -0.009268548, -0.11880...","[-0.046399742, -0.33773792, 0.7608252, 0.47063...","[[0.6121709942817688, 0.28514373302459717, 1.2...","[0.612874, 0.010994982, -0.65460473, 0.1742316...","[-0.528413712978363, 0.19533416628837585, 0.65...","[0.20099634, -0.20690696, 0.08150868, 0.554392...",...,1,0,0.157232,NaN,3.224354e-02,2.075149e-09,0.000000,42.001875,1.956062e-01,0.012688
1,0,"[-0.20527962, 0.35404864, -0.7783211, 0.594439...",None,"[1.9521571, -0.45306733, 0.029711116, -0.27353...","[-0.22496563, -0.23461707, 0.09076467, 0.43862...","[2.0140765, -0.4254004, 0.056320373, -0.372034...","[[0.3260622024536133, -0.19332827627658844, -0...","[0.4628146, 0.17595449, -0.18368044, 0.39444, ...","[-0.5505270957946777, 0.5257819294929504, 0.47...","[-0.22318864, 0.33674824, -0.7702154, 0.579116...",...,1,1,0.000770,NaN,0.000000e+00,0.000000e+00,0.000000,9.780028,2.910004e-09,0.013775
2,0,"[-0.43341294, 0.23154245, -0.8743953, 0.638324...","[0.4401156, -0.21285827, 0.5240711, -0.0887983...","[1.9349287, -0.053028245, 0.066755235, 0.04048...","[0.70692855, 1.0216618, 1.3932865, 0.18482241,...","[1.1919316, -0.49624836, -0.19844365, 0.175008...","[[-0.11487230658531189, 0.4011990427970886, 0....","[0.49475476, 0.14105393, -0.59882516, -0.03755...","[-0.6331321001052856, 0.47984400391578674, 0.5...","[-0.4563888, 0.22507098, -0.91505283, 0.621529...",...,1,1,0.000381,4.989550e-09,1.935401e-08,0.000000e+00,0.000000,2.725666,0.000000e+00,0.000000
3,0,"[-0.31406865, 0.031150397, -0.8103627, 0.34940...","[1.3241816, -0.20996426, 0.41126335, 0.0240248...","[0.9589939, -1.0113164, 0.50199246, 0.02142361...","[-0.22496563, -0.23461707, 0.09076467, 0.43862...","[0.009384095, -0.6564601, 0.8625589, 0.0680266...","[[0.8406607508659363, -0.6243898868560791, -0....","[-0.23589465, 0.067371085, -0.7356764, 0.02474...","[-0.37794095277786255, 0.5866395235061646, 0.5...","[-0.3634312, 0.1448457, -0.85151, 0.43155274, ...",...,1,1,0.006625,2.202066e-08,0.000000e+00,0.000000e+00,0.000000,14.575337,8.735670e-03,0.021237
4,0,"[-0.38587776, 0.2615051, -0.44857445, 0.784297...","[1.5472196, -0.05469868, 0.12639447, -0.004592...",None,"[1.8150815, -0.21320908, 0.27021378, -0.101866...","[1.9107708, -0.40683055, 0.07942994, -0.296098...","[[0.229730486869812, 0.03908279538154602, 0.71...","[-0.29257202, 0.29213133, -1.2101544, 0.630881...","[-0.5304545164108276, 0.4053342044353485, 0.51...","[-0.57106173, 0.17932498, -0.6860113, 0.789167...",...,0,1,0.024320,0.000000e+00,NaN,4.993930e-09,0.037286,29.796113,3.084421e-01,0.039195


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2500 entries, 0 to 1201
Data columns (total 27 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   target                              2500 non-null   int64  
 1   characteristic_attributes_mapping1  2500 non-null   object 
 2   color1                              1561 non-null   object 
 3   type1                               2385 non-null   object 
 4   cat21                               2500 non-null   object 
 5   cat41                               2500 non-null   object 
 6   main_pic_embeddings_resnet_v11      2500 non-null   object 
 7   description1                        2224 non-null   object 
 8   name_bert_641                       2500 non-null   object 
 9   characteristic_attributes_mapping2  2500 non-null   object 
 10  color2                              1556 non-null   object 
 11  type2                               2387 non-nul

In [7]:
target = train_df['target']
features = train_df.drop('target', axis=1)

In [8]:
features['main_pic_embeddings_resnet_v11'] = features['main_pic_embeddings_resnet_v11'].apply(lambda x: x[0])
features['main_pic_embeddings_resnet_v12'] = features['main_pic_embeddings_resnet_v12'].apply(lambda x: x[0])

In [9]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2500 entries, 0 to 1201
Data columns (total 26 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   characteristic_attributes_mapping1  2500 non-null   object 
 1   color1                              1561 non-null   object 
 2   type1                               2385 non-null   object 
 3   cat21                               2500 non-null   object 
 4   cat41                               2500 non-null   object 
 5   main_pic_embeddings_resnet_v11      2500 non-null   object 
 6   description1                        2224 non-null   object 
 7   name_bert_641                       2500 non-null   object 
 8   characteristic_attributes_mapping2  2500 non-null   object 
 9   color2                              1556 non-null   object 
 10  type2                               2387 non-null   object 
 11  cat22                               2500 non-nul

In [10]:
features.loc[features['color1'].isna(), 'color1'] = features['color1'][features['color1'].isna()].apply(lambda x: np.empty(0))
features.loc[features['description1'].isna(), 'description1'] = features['description1'][features['description1'].isna()].apply(lambda x: np.empty(0))
features.loc[features['type1'].isna(), 'type1'] = features['type1'][features['type1'].isna()].apply(lambda x: np.empty(0))

features.loc[features['color2'].isna(), 'color2'] = features['color2'][features['color2'].isna()].apply(lambda x: np.empty(0))
features.loc[features['description2'].isna(), 'description2'] = features['description2'][features['description2'].isna()].apply(lambda x: np.empty(0))
features.loc[features['type2'].isna(), 'type2'] = features['type2'][features['type2'].isna()].apply(lambda x: np.empty(0))

In [11]:
features_columns = list(features.columns)
emb_features = [features_columns[i] for i in range(0, 16)]

for emb_col in emb_features:
    features[emb_col] = features[emb_col].apply(lambda x: x.astype(np.float32) if x is not None else np.zeros(128))

In [12]:
features = features.drop(['color1', 'color2', 'type1', 'type2', 'description1', 'description2'], axis=1)

In [13]:
features

,characteristic_attributes_mapping1,cat21,cat41,main_pic_embeddings_resnet_v11,name_bert_641,characteristic_attributes_mapping2,cat22,cat42,main_pic_embeddings_resnet_v12,name_bert_642,brand_eq,country_eq,attr_dis,color_dis,type_dis,cat2_dis,cat4_dis,pic_dis,description_dis,name_dis
0,"[-0.43684134, 0.12223242, -0.37675297, 0.76652...","[2.0046031, -0.5959641, -0.009268548, -0.11880...","[-0.046399742, -0.33773792, 0.7608252, 0.47063...","[0.612171, 0.28514373, 1.2190557, 1.0417652, -...","[-0.5284137, 0.19533417, 0.653045, -0.4589088,...","[0.20099634, -0.20690696, 0.08150868, 0.554392...","[2.0046031, -0.5959641, -0.009268548, -0.11880...","[-0.046399742, -0.33773792, 0.7608252, 0.47063...","[0.24847914, 0.07382858, 1.2195343, -0.2660364...","[-0.3290029, 0.20316873, 0.57139236, -0.444630...",1,0,1.572316e-01,NaN,3.224354e-02,2.075149e-09,0.000000e+00,42.001875,1.956062e-01,0.012688
1,"[-0.20527962, 0.35404864, -0.7783211, 0.594439...","[-0.22496563, -0.23461707, 0.09076467, 0.43862...","[2.0140765, -0.4254004, 0.056320373, -0.372034...","[0.3260622, -0.19332828, -0.10404004, -0.45531...","[-0.5505271, 0.5257819, 0.47485995, -0.4253559...","[-0.22318864, 0.33674824, -0.7702154, 0.579116...","[-0.22496563, -0.23461707, 0.09076467, 0.43862...","[2.0140765, -0.4254004, 0.056320373, -0.372034...","[0.47268084, -0.25320494, 0.08988632, -0.48186...","[-0.4999013, 0.5409635, 0.427443, -0.36424908,...",1,1,7.701501e-04,NaN,0.000000e+00,0.000000e+00,0.000000e+00,9.780028,2.910004e-09,0.013775
2,"[-0.43341294, 0.23154245, -0.8743953, 0.638324...","[0.70692855, 1.0216618, 1.3932865, 0.18482241,...","[1.1919316, -0.49624836, -0.19844365, 0.175008...","[-0.11487231, 0.40119904, 0.33857632, 0.335929...","[-0.6331321, 0.479844, 0.5797135, -0.5452931, ...","[-0.4563888, 0.22507098, -0.91505283, 0.621529...","[0.70692855, 1.0216618, 1.3932865, 0.18482241,...","[1.1919316, -0.49624836, -0.19844365, 0.175008...","[-0.09815172, 0.43483564, 0.3523121, 0.3802402...","[-0.6331321, 0.479844, 0.5797135, -0.5452931, ...",1,1,3.806716e-04,4.989550e-09,1.935401e-08,0.000000e+00,0.000000e+00,2.725666,0.000000e+00,0.000000
3,"[-0.31406865, 0.031150397, -0.8103627, 0.34940...","[-0.22496563, -0.23461707, 0.09076467, 0.43862...","[0.009384095, -0.6564601, 0.8625589, 0.0680266...","[0.84066075, -0.6243899, -0.5123942, -0.836276...","[-0.37794095, 0.5866395, 0.5182769, -0.3534220...","[-0.3634312, 0.1448457, -0.85151, 0.43155274, ...","[-0.22496563, -0.23461707, 0.09076467, 0.43862...","[0.009384095, -0.6564601, 0.8625589, 0.0680266...","[0.8156735, -0.7285702, -0.4726927, -0.9229734...","[-0.45659527, 0.500046, 0.6082737, -0.42728546...",1,1,6.624993e-03,2.202066e-08,0.000000e+00,0.000000e+00,0.000000e+00,14.575337,8.735670e-03,0.021237
4,"[-0.38587776, 0.2615051, -0.44857445, 0.784297...","[1.8150815, -0.21320908, 0.27021378, -0.101866...","[1.9107708, -0.40683055, 0.07942994, -0.296098...","[0.22973049, 0.039082795, 0.7120164, 0.5068465...","[-0.5304545, 0.4053342, 0.51110256, -0.5279125...","[-0.57106173, 0.17932498, -0.6860113, 0.789167...","[1.8150815, -0.21320908, 0.27021378, -0.101866...","[1.7156291, 0.21666233, -0.19947597, -0.140445...","[0.50200975, -0.019166484, 1.0085977, 0.692578...","[-0.42492023, 0.5134461, 0.61161804, -0.147389...",0,1,2.432034e-02,0.000000e+00,NaN,4.993930e-09,3.728641e-02,29.796113,3.084421e-01,0.039195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197,"[-0.47388014, 0.30103526, -0.6238369, 0.630042...","[1.310419, -0.4621484, 1.1588186, 0.04485536, ...","[1.0063177, 0.24485098, 0.70466363, 0.35587564...","[0.23224483, -0.3795752, -0.4074942, 0.5386239...","[-0.41782212, 0.31494492, 0.7188773, -0.795285...","[-0.3317828, 0.33893508, -0.7218484, 0.4750344...","[1.310419, -0.4621484, 1.1588186, 0.04485536, ...","[1.0063177, 0.24485098, 0.70466363, 0.35587564...","[0.23224483, -0.3795752, -0.4074942, 0.5386239...","[-0.41782212, 0.31494492, 0.7188773, -0.795285...",1,1,4.718542e-03,0.00

In [14]:
X_train, X_val, y_train, y_val = train_test_split(features, target, random_state=42, train_size=0.7, shuffle=True, stratify=target)

In [15]:
X_train.shape, X_val.shape

((1750, 20), (750, 20))

In [17]:
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, train_size=0.7, random_state=42, shuffle=True, stratify=y_val)

In [20]:
X_train

,characteristic_attributes_mapping1,cat21,cat41,main_pic_embeddings_resnet_v11,name_bert_641,characteristic_attributes_mapping2,cat22,cat42,main_pic_embeddings_resnet_v12,name_bert_642,brand_eq,country_eq,attr_dis,color_dis,type_dis,cat2_dis,cat4_dis,pic_dis,description_dis,name_dis
152,"[-0.26642373, 0.13179034, -0.8009117, 0.303580...","[1.5279986, -0.4267642, -0.025356855, -0.37532...","[1.7247863, -0.010899592, 0.030462865, -0.6666...","[0.40294585, 1.6207253, -0.23309773, -0.521838...","[-0.621788, 0.6695777, 0.61766326, -0.2954334,...","[-0.26642373, 0.13179034, -0.8009117, 0.303580...","[1.5279986, -0.4267642, -0.025356855, -0.37532...","[1.7247863, -0.010899592, 0.030462865, -0.6666...","[0.40294585, 1.6207253, -0.23309773, -0.521838...","[-0.621788, 0.6695777, 0.61766326, -0.2954334,...",1,0,0.000000e+00,1.572676e-09,1.458277e-08,3.273221e-09,1.565454e-09,0.000000,0.000000e+00,0.000000
599,"[-0.35724208, 0.109075904, -0.5089222, 0.46608...","[1.5322148, -0.21915339, 0.6406332, -0.1343247...","[0.6333035, -1.1935614, 0.32802075, -0.3972359...","[1.0090547, 1.149305, 0.66672915, 1.1645404, 1...","[-0.40236396, 0.52142894, 0.42415443, -0.37609...","[-0.35724208, 0.109075904, -0.5089222, 0.46608...","[1.5322148, -0.21915339, 0.6406332, -0.1343247...","[0.6333035, -1.1935614, 0.32802075, -0.3972359...","[1.0090547, 1.149305, 0.66672915, 1.1645404, 1...","[-0.40236396, 0.52142894, 0.42415443, -0.37609...",1,1,0.000000e+00,0.000000e+00,0.000000e+00,4.691390e-09,1.313200e-08,0.000000,5.748723e-09,0.000000
694,"[-0.35267904, 0.18441777, -0.3306691, 0.549161...","[1.8150815, -0.21320908, 0.27021378, -0.101866...","[1.6252887, 0.14859551, -0.13218385, -0.225070...","[1.1097257, 0.02569762, 0.06367546, 0.00085899...","[-0.5881848, 0.60949796, 0.41653752, -0.559725...","[-0.35267904, 0.18441777, -0.3306691, 0.549161...","[1.8150815, -0.21320908, 0.27021378, -0.101866...","[1.6252887, 0.14859551, -0.13218385, -0.225070...","[1.1097257, 0.02569762, 0.06367546, 0.00085899...","[-0.5881848, 0.60949796, 0.41653752, -0.559725...",1,1,0.000000e+00,0.000000e+00,NaN,4.993930e-09,1.543192e-08,0.000000,3.370242e-09,0.000000
794,"[-0.2120854, 0.22318725, -0.8250926, 0.7148825...","[0.70692855, 1.0216618, 1.3932865, 0.18482241,...","[1.0572565, -0.6431988, 0.6512874, 0.43212572,...","[0.2712882, -0.06112492, 0.8743601, 0.5828197,...","[-0.54663557, 0.6457303, 0.5085628, -0.5742468...","[-0.438817, 0.2011831, -0.654438, 0.35099432, ...","[0.70692855, 1.0216618, 1.3932865, 0.18482241,...","[1.0572565, -0.6431988, 0.6512874, 0.43212572,...","[0.2712882, -0.06112492, 0.8743601, 0.5828197,...","[-0.5673691, 0.66160095, 0.4720597, -0.5691403...",1,1,4.033010e-02,0.000000e+00,0.000000e+00,0.000000e+00,5.550242e-09,0.000000,6.045080e-02,0.003757
855,"[-0.23911716, -0.22727801, -1.0645311, 0.09944...","[0.66388637, -0.5674183, 0.40921578, -0.505234...","[0.32127574, -0.1572261, -0.16480449, 0.442679...","[-0.008134402, -0.14177671, -0.08354104, 0.101...","[-0.4689821, 0.42844746, 0.50786215, -0.576173...","[-0.23961253, -0.20907697, -0.8914128, 0.11303...","[0.66388637, -0.5674183, 0.40921578, -0.505234...","[0.32127574, -0.1572261, -0.16480449, 0.442679...","[-0.008134402, -0.14177671, -0.08354104, 0.101...","[-0.46147105, 0.52293545, 0.40106502, -0.55729...",1,1,4.372072e-03,1.874490e-08,0.000000e+00,1.862459e-08,0.000000e+00,0.000000,4.420315e-03,0.007982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187,"[-0.40944365, 0.25699013, -0.6197277, 0.612077...","[1.8150815, -0.21320908, 0.27021378, -0.101866...","[1.3937305, -0.5287866, 0.6968599, -0.13077058...","[0.95687604, 0.47239774, 0.19812964, 0.2622182...","[-0.38347387, 0.63901687, 0.39834258, -0.48073...","[-0.40944365, 0.25699013, -0.6197277, 0.612077...","[1.8150815, -0.21320908, 0.27021378, -0.101866...","[1.3937305, -0.5287866, 0.6968599, -0.13077058...","[0.30729324, 0.50473475, 0.049800724, -0.02291...","[-0.38347387, 0.63901687, 0.39834258, -0.48073.

In [22]:
emb_features_after_preprocessing = ['characteristic_attributes_mapping1','cat21','cat41','main_pic_embeddings_resnet_v11','name_bert_641','characteristic_attributes_mapping2','cat22','cat42','main_pic_embeddings_resnet_v12','name_bert_642']

In [24]:
train_data = catboost.Pool(data=X_train, label=y_train, embedding_features=emb_features_after_preprocessing)
val_data = catboost.Pool(data=X_val, label=y_val, embedding_features=emb_features_after_preprocessing)
test_data = catboost.Pool(data=X_test, label=y_test, embedding_features=emb_features_after_preprocessing)

In [27]:
def objective(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'boosting_type': trial.suggest_categorical('boosting_type', ['Ordered', 'Plain']),
    }

    model = catboost.CatBoostClassifier(**params, random_seed=42)
    model.fit(train_data, verbose=3, eval_set=val_data)
    y_pred = model.predict(val_data)
    return eval_metric(val_data.get_label(), y_pred, 'Precision')

sampler = TPESampler(seed=41)
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=20)

[I 2024-11-15 18:56:15,934] A new study created in memory with name: no-name-ae42fa4b-c06a-4656-960d-29faf240c976


0:	learn: 0.6824031	test: 0.6823048	best: 0.6823048 (0)	total: 1.86ms	remaining: 1.86s
3:	learn: 0.6496777	test: 0.6497060	best: 0.6497060 (3)	total: 7.37ms	remaining: 1.84s
6:	learn: 0.6273292	test: 0.6258438	best: 0.6258438 (6)	total: 12.2ms	remaining: 1.74s
9:	learn: 0.6030920	test: 0.6037900	best: 0.6037900 (9)	total: 17.3ms	remaining: 1.71s
12:	learn: 0.5817091	test: 0.5841187	best: 0.5841187 (12)	total: 22.2ms	remaining: 1.68s
15:	learn: 0.5657534	test: 0.5667292	best: 0.5667292 (15)	total: 27.2ms	remaining: 1.67s
18:	learn: 0.5515155	test: 0.5527533	best: 0.5527533 (18)	total: 32.6ms	remaining: 1.68s
21:	learn: 0.5400342	test: 0.5419284	best: 0.5419284 (21)	total: 37.6ms	remaining: 1.67s
24:	learn: 0.5311602	test: 0.5342616	best: 0.5342616 (24)	total: 42.5ms	remaining: 1.66s
27:	learn: 0.5220510	test: 0.5254728	best: 0.5254728 (27)	total: 47.5ms	remaining: 1.65s
30:	learn: 0.5125723	test: 0.5169003	best: 0.5169003 (30)	total: 52.3ms	remaining: 1.64s
33:	learn: 0.5054535	test: 0.

[I 2024-11-15 18:56:27,762] Trial 0 finished with value: 0.826271186440678 and parameters: {'learning_rate': 0.032583126137044616, 'depth': 3, 'l2_leaf_reg': 7.091346169995026, 'boosting_type': 'Plain'}. Best is trial 0 with value: 0.826271186440678.


0:	learn: 0.6666248	test: 0.6673710	best: 0.6673710 (0)	total: 9.48ms	remaining: 9.47s
3:	learn: 0.6060496	test: 0.6105896	best: 0.6105896 (3)	total: 30ms	remaining: 7.48s
6:	learn: 0.5591390	test: 0.5653064	best: 0.5653064 (6)	total: 48.3ms	remaining: 6.85s
9:	learn: 0.5208207	test: 0.5314397	best: 0.5314397 (9)	total: 66.9ms	remaining: 6.62s
12:	learn: 0.4980441	test: 0.5124990	best: 0.5124990 (12)	total: 85ms	remaining: 6.45s
15:	learn: 0.4821428	test: 0.5004866	best: 0.5004866 (15)	total: 103ms	remaining: 6.36s
18:	learn: 0.4694212	test: 0.4898908	best: 0.4898908 (18)	total: 122ms	remaining: 6.29s
21:	learn: 0.4584347	test: 0.4814652	best: 0.4814652 (21)	total: 141ms	remaining: 6.26s
24:	learn: 0.4511880	test: 0.4763251	best: 0.4763251 (24)	total: 159ms	remaining: 6.2s
27:	learn: 0.4408010	test: 0.4691674	best: 0.4691674 (27)	total: 177ms	remaining: 6.16s
30:	learn: 0.4322241	test: 0.4629879	best: 0.4629879 (30)	total: 195ms	remaining: 6.11s
33:	learn: 0.4270122	test: 0.4592630	bes

[I 2024-11-15 18:56:52,177] Trial 1 finished with value: 0.8247863247863247 and parameters: {'learning_rate': 0.06434791200539318, 'depth': 4, 'l2_leaf_reg': 7.016641455343112, 'boosting_type': 'Ordered'}. Best is trial 0 with value: 0.826271186440678.


...

[I 2024-11-15 19:14:26,268] Trial 17 finished with value: 0.8283261802575107 and parameters: {'learning_rate': 0.023801521782827604, 'depth': 9, 'l2_leaf_reg': 1.4690567145205486, 'boosting_type': 'Ordered'}. Best is trial 8 with value: 0.8318965517241379.


0:	learn: 0.6566441	test: 0.6606804	best: 0.6606804 (0)	total: 16.3ms	remaining: 16.3s
3:	learn: 0.5830968	test: 0.5915549	best: 0.5915549 (3)	total: 67.8ms	remaining: 16.9s
6:	learn: 0.5384191	test: 0.5502527	best: 0.5502527 (6)	total: 119ms	remaining: 16.9s
9:	learn: 0.4974256	test: 0.5157612	best: 0.5157612 (9)	total: 171ms	remaining: 17s
12:	learn: 0.4729198	test: 0.4989525	best: 0.4989525 (12)	total: 224ms	remaining: 17s
15:	learn: 0.4453656	test: 0.4764450	best: 0.4764450 (15)	total: 277ms	remaining: 17s
18:	learn: 0.4306787	test: 0.4644568	best: 0.4644568 (18)	total: 324ms	remaining: 16.7s
21:	learn: 0.4167503	test: 0.4562052	best: 0.4562052 (21)	total: 376ms	remaining: 16.7s
24:	learn: 0.4058271	test: 0.4517903	best: 0.4517903 (24)	total: 429ms	remaining: 16.7s
27:	learn: 0.3975079	test: 0.4474060	best: 0.4474060 (27)	total: 480ms	remaining: 16.7s
30:	learn: 0.3901540	test: 0.4439981	best: 0.4439981 (30)	total: 521ms	remaining: 16.3s
33:	learn: 0.3854716	test: 0.4412706	best: 0

[I 2024-11-15 19:15:02,171] Trial 18 finished with value: 0.8405172413793104 and parameters: {'learning_rate': 0.07305450041983286, 'depth': 6, 'l2_leaf_reg': 4.032467484366716, 'boosting_type': 'Ordered'}. Best is trial 18 with value: 0.8405172413793104.


0:	learn: 0.6609752	test: 0.6624684	best: 0.6624684 (0)	total: 15.5ms	remaining: 15.5s
3:	learn: 0.5817158	test: 0.5890566	best: 0.5890566 (3)	total: 65.8ms	remaining: 16.4s
6:	learn: 0.5304909	test: 0.5410077	best: 0.5410077 (6)	total: 116ms	remaining: 16.5s
9:	learn: 0.4941949	test: 0.5082917	best: 0.5082917 (9)	total: 167ms	remaining: 16.5s
12:	learn: 0.4695510	test: 0.4908916	best: 0.4908916 (12)	total: 219ms	remaining: 16.6s
15:	learn: 0.4488846	test: 0.4720038	best: 0.4720038 (15)	total: 270ms	remaining: 16.6s
18:	learn: 0.4347255	test: 0.4631363	best: 0.4631363 (18)	total: 320ms	remaining: 16.5s
21:	learn: 0.4244803	test: 0.4556644	best: 0.4556644 (21)	total: 359ms	remaining: 16s
24:	learn: 0.4143942	test: 0.4490244	best: 0.4490244 (24)	total: 410ms	remaining: 16s
27:	learn: 0.4057736	test: 0.4434684	best: 0.4434684 (27)	total: 463ms	remaining: 16.1s
30:	learn: 0.3980545	test: 0.4391400	best: 0.4391400 (30)	total: 497ms	remaining: 15.6s
33:	learn: 0.3913448	test: 0.4350964	best:

[I 2024-11-15 19:15:37,883] Trial 19 finished with value: 0.825531914893617 and parameters: {'learning_rate': 0.07688916849440071, 'depth': 6, 'l2_leaf_reg': 9.853373162518773, 'boosting_type': 'Ordered'}. Best is trial 18 with value: 0.8405172413793104.


In [38]:
def calc_test_quality(train_pool=train_data, val_pool=val_data, test_pool=test_data, **kwargs):
    print(kwargs)
    model = catboost.CatBoostClassifier(**kwargs, random_seed=42)
    model.fit(train_data, verbose=5, eval_set=val_data)
    y_pred = model.predict(test_pool)
    return eval_metric(test_pool.get_label(), y_pred, 'AUC'), eval_metric(test_pool.get_label(), y_pred, 'Recall'), eval_metric(test_pool.get_label(), y_pred, 'Accuracy') 

In [29]:
best_model_kwrgs={'learning_rate': 0.07305450041983286, 'depth': 6, 'l2_leaf_reg': 4.032467484366716, 'boosting_type': 'Ordered'}

In [35]:
calc_test_quality(**best_model_kwargs)

{'learning_rate': 0.07305450041983286, 'depth': 6, 'l2_leaf_reg': 4.032467484366716, 'boosting_type': 'Ordered'}


([0.8568376068376069], [0.8333333333333334], [0.8577777777777778])